## Build a GenAI Application
This is the continuation of the part1. In Part1, we persisted the embeddings in a vectordB in the local disk. In the Part2, we will query the embeddings using llm, and chains.

```
User Query
   ↓
Retriever Chain → Documents
   ↓
Document Chain → LLM Answer

```

We will learn 2 chains here:

1. Retriever Chain: A Retriever Chain is responsible for getting relevant documents from a knowledge base (vector store, keyword search, etc.) in response to a user query. 

```
User Query → Retriever → Relevant Documents
```

2. Document Chain: A Document Chain (aka QA chain or Answer-generating chain) takes the documents retrieved (e.g., from a Retriever) and uses an LLM to synthesize an answer.

```
Query + Retrieved Docs → LLM → Final Answer

```


In [ ]:
## Read Environment Variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
## Step0: Initialize the LLM Model
from utility.llm_factory import LLMFactory
llm = LLMFactory.get_llm('openai')

from utility.embedding_factory import EmbeddingFactory

embedding_model = EmbeddingFactory.get_llm('openai')
embedding_model

In [ ]:
## Step1: Load the vector store from the persisted directory
from langchain.vectorstores import Chroma

## Step 1: Load the vector store from the persisted directory
vector_store_db_loaded = Chroma(    
    collection_name="ragtest1_collection",
    embedding_function=embedding_model,
    persist_directory="./_data/chroma_db"
)

In [ ]:
## Step2: Create a retriever from the vector store
retriever = vector_store_db_loaded.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [ ]:
## Step 3: Create a Dcoument Chain for Question Answering 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the provided context:
    <context>{context}</context>
    Question: {input}
    """
)

# link the LLM and the prompt to create a document chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_variable_name="context"
)

document_chain

In [ ]:
## Step 4: Create the Retriever Chain (by linking the retriever and document chain)

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)
print(retriever_chain)

result = retriever_chain.invoke({
    "input": "How to find my current usages?",
    "chat_history": []
})
print(result['answer'])

In [10]:
## Let's Query the retriever chain with a question
# response = retriever_chain.invoke({"input":"How to find my current usages?"})
response = retriever_chain.invoke({"input":"How to change the retention defaults?"})
response["answer"]

'To change the organization level retention defaults, navigate to the Usage configuration tab and modify the settings. Bear in mind, this affects all new projects. For changing project level retention defaults, primarily for pre-existing projects, go to "Projects", select your project name, click the data retention drop-down, and modify it to base retention. This will affect retention and pricing for traces going forward.'